In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
images_dir = '/kaggle/input/food-101/food-101/food-101/images/'
train_meta_dir = '/kaggle/input/food-101/food-101/food-101/meta/train.txt'
test_meta_dir = '/kaggle/input/food-101/food-101/food-101/meta/test.txt'

def prepare_data(meta_dir, images_dir, dest_dir):
    from collections import defaultdict
    import os
    from shutil import copy

    classes_images_dict = defaultdict(list)
    with open(meta_dir, 'r') as txt:
        paths = [line.strip() for line in txt.readlines()]
        for path in paths:
            food = path.split('/')
            classes_images_dict[food[0]].append(food[1] + '.jpg')

    for food in classes_images_dict.keys():
        food_dest_dir = os.path.join(dest_dir, food)
        if not os.path.exists(food_dest_dir):
            os.makedirs(food_dest_dir)
        for img in classes_images_dict[food]:
            src_path = os.path.join(images_dir, food, img)
            img_dest_path = os.path.join(food_dest_dir, img)
            if os.path.exists(src_path):
                copy(src_path, img_dest_path)

prepare_data(train_meta_dir, images_dir, 'train')
prepare_data(test_meta_dir, images_dir, 'test')

In [ ]:
image_size = (224, 224)  

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, #densenet preprocessing
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

training_dir = 'train'
validation_dir = 'test'
batch_size = 32  
num_classes = 101
training_set_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_set_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

training_set = tf.data.Dataset.from_generator(
    lambda: training_set_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(
        (None, image_size[0], image_size[1], 3),
        (None, num_classes),
    )
    ).repeat().prefetch(tf.data.AUTOTUNE)

validation_set = tf.data.Dataset.from_generator(
    lambda: validation_set_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(
        (None, image_size[0], image_size[1], 3),
        (None, num_classes)
    )
).repeat().prefetch(tf.data.AUTOTUNE)

steps_per_epoch = len(training_set_generator)
validation_steps = len(validation_set_generator)

In [ ]:
model_checkpoint = ModelCheckpoint(
    f"best_densenet_model.keras", 
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

early_stopping = EarlyStopping(
    patience=10,  
    monitor='val_loss',
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

In [ ]:
def build_densenet_model():
    base_model = DenseNet201(
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )
    
    for layer in base_model.layers[:-50]:
        layer.trainable = False
    
    inputs = base_model.input
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(256, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    outputs = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

model = build_densenet_model()
    
model.compile(
    optimizer=Adam(learning_rate=1e-4), 
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    ax1.plot(history.history['accuracy'], label='Training accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation accuracy')
    ax1.set_title('Model accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    ax2.plot(history.history['loss'], label='Training loss')
    ax2.plot(history.history['val_loss'], label='Validation loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=50,  
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint, early_stopping, reduce_lr]
)

plot_training_history(history)

In [ ]:
final_loss, final_accuracy = model.evaluate(
    validation_set,
    steps=validation_steps
)
print(f"Validation loss: {final_loss:.4f}")
print(f"Validation accuracy: {final_accuracy:.4f}")

model.save('final_densenet201_food101_model.keras')